In [1]:
import random
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
path='./case1/1.xlsx'
Job_pd=pd.read_excel(path,sheet_name='Job info')
Mac_pd=pd.read_excel(path,sheet_name='Machine info')
Pr1_pd=pd.read_excel(path,sheet_name='Process 1 Time')
Pr2_pd=pd.read_excel(path,sheet_name='Process 2 Time')
# 定义问题的参数
J = len(Job_pd)
M = len(Mac_pd)
W = Job_pd['准备时间'].mean()  # 订单的 Task0 和 Task1 之间的准备时间
N = Job_pd['需求量'].values  # 订单的需求量
A = Job_pd['到达时刻'].values  # 订单的到达时刻
D = Job_pd['交货期'].values  # 订单的交货期
pt= np.array([Pr1_pd[['Machine_0','Machine_1']].values,Pr2_pd[['Machine_0','Machine_1']].values])# 订单的 Task0 在生产线 i 上的单位处理时长 # 订单的 Task1 在生产线 i 上的单位处理时长
p = np.array([Mac_pd[['闲置功率','工作功率']].values[:,0],Mac_pd[['闲置功率','工作功率']].values[:,1]])   # 生产线的闲置功率和工作功率
rawdata=pd.concat([Job_pd,Pr1_pd[['Machine_0','Machine_1']],Pr2_pd[['Machine_0','Machine_1']]],axis=1).iloc[:,1:]

In [2]:
rawdata

,准备时间,需求量,到达时刻,交货期,Machine_0,Machine_1,Machine_0,Machine_1
0,4320,760,1872,29232,11,7,13,7
1,4320,450,3744,29664,5,8,14,11
2,4320,60,3168,30528,6,7,7,12
3,4320,220,1584,23184,8,7,14,12
4,4320,400,1152,24192,12,8,11,10
...,...,...,...,...,...,...,...,...
195,4320,480,3456,29376,9,11,13,13
196,4320,640,3024,30384,10,6,14,11
197,4320,550,576,22176,11,6,12,11
198,4320,690,3024,27504,8,6,9,8


In [4]:
rawdata.describe()

,准备时间,需求量,到达时刻,交货期,Machine_0,Machine_1,Machine_0,Machine_1
count,200.0,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,4320.0,518.55000,1942.560000,26494.560000,8.460000,8.505000,10.500000,10.645000
std,0.0,271.35135,1280.916446,2564.151503,2.329276,2.210067,2.331777,2.230212
min,4320.0,50.00000,0.000000,21600.000000,5.000000,5.000000,7.000000,7.000000
25%,4320.0,277.50000,720.000000,24624.000000,6.000000,7.000000,8.000000,9.000000
50%,4320.0,540.00000,1872.000000,26496.000000,8.000000,8.000000,11.000000,11.000000
75%,4320.0,750.00000,3024.000000,28260.000000,10.250000,10.000000,13.000000,13.000000
max,4320.0,980.00000,4176.000000,31536.000000,12.000000,12.000000,14.000000,14.000000


In [2]:
rawdata=pd.concat([Job_pd,Pr1_pd[['Machine_0','Machine_1']],Pr2_pd[['Machine_0','Machine_1']]],axis=1).iloc[:,1:]
column_length=len(rawdata.columns)
# 创建一个新的行，所有值均为0
zero_row = pd.Series([0] * column_length, index=rawdata.columns)
# 将新行附加到DataFrame末尾
dsize=len(rawdata)
for i in range(200-dsize):
    rawdata = rawdata.append(zero_row, ignore_index=True)
rawdata=pd.concat([rawdata,rawdata.describe()])
rawdata = rawdata.append(pd.Series(p.flatten().tolist()+[0] * (column_length-4), index=rawdata.columns), ignore_index=True)
rawdata

,准备时间,需求量,到达时刻,交货期,Machine_0,Machine_1,Machine_0,Machine_1
0,4320.0,760.0,1872.0,29232.0,11.00,7.0,13.0,7.0
1,4320.0,450.0,3744.0,29664.0,5.00,8.0,14.0,11.0
2,4320.0,60.0,3168.0,30528.0,6.00,7.0,7.0,12.0
3,4320.0,220.0,1584.0,23184.0,8.00,7.0,14.0,12.0
4,4320.0,400.0,1152.0,24192.0,12.00,8.0,11.0,10.0
...,...,...,...,...,...,...,...,...
204,4320.0,277.5,720.0,24624.0,6.00,7.0,8.0,9.0
205,4320.0,540.0,1872.0,26496.0,8.00,8.0,11.0,11.0
206,4320.0,750.0,3024.0,28260.0,10.25,10.0,13.0,13.0
207,4320.0,980.0,4176.0,31536.0,12.00,12.0,14.0,14.0


In [3]:
def env2vec(path):
    Job_pd=pd.read_excel(path,sheet_name='Job info')
    Mac_pd=pd.read_excel(path,sheet_name='Machine info')
    Pr1_pd=pd.read_excel(path,sheet_name='Process 1 Time')
    Pr2_pd=pd.read_excel(path,sheet_name='Process 2 Time')
    p = np.array([Mac_pd[['闲置功率','工作功率']].values[:,0],Mac_pd[['闲置功率','工作功率']].values[:,1]])   # 生产线的闲置功率和工作功率
    rawdata=pd.concat([Job_pd,Pr1_pd[['Machine_0','Machine_1']],Pr2_pd[['Machine_0','Machine_1']]],axis=1).iloc[:,1:]
    column_length=len(rawdata.columns)
    # 创建一个新的行，所有值均为0
    zero_row = pd.Series([0] * column_length, index=rawdata.columns)
    # 将新行附加到DataFrame末尾
    dsize=len(rawdata)
    for i in range(200-dsize):
        rawdata = rawdata.append(zero_row, ignore_index=True)
    rawdata=pd.concat([rawdata,rawdata.describe()])
    rawdata = rawdata.append(pd.Series(p.flatten().tolist()+[0] * (column_length-4), index=rawdata.columns), ignore_index=True)
    return rawdata.to_numpy()

### 归一化

In [4]:
import os
def find_image_file(source_path, file_lst):
    """
    递归寻找 文件夹以及子目录的 图片文件。
    :param source_path: 源文件夹路径
    :param file_lst: 输出 文件路径列表
    :return:
    """
    image_ext = ['.xlsx']
    for dir_or_file in os.listdir(source_path):
        file_path = os.path.join(source_path, dir_or_file)
        if os.path.isfile(file_path):  # 判断是否为文件
            file_name_ext = os.path.splitext(os.path.basename(file_path))  # 文件名与后缀
            if len(file_name_ext) < 2:
                continue
            if file_name_ext[1] in image_ext:  # 后缀在后缀列表中
                file_lst.append(file_path)
            else:
                continue
        elif os.path.isdir(file_path):  # 如果是个dir，则再次调用此函数，传入当前目录，递归处理。
            find_image_file(file_path, file_lst)
        else:
            print('文件夹没有环境' + os.path.basename(file_path))
env_path_list=[]
find_image_file('./',env_path_list)
labels=[int(x[x.find('case')+4:x.rfind('/')]) for x in env_path_list]
env_path=np.array([env2vec(x) for x in env_path_list])
env_path

array([[[5.760e+03, 5.800e+02, 4.176e+03, ..., 6.000e+00, 8.000e+00,
         9.000e+00],
        [5.760e+03, 8.500e+02, 1.728e+03, ..., 1.200e+01, 1.400e+01,
         8.000e+00],
        [5.760e+03, 7.200e+02, 3.744e+03, ..., 1.100e+01, 1.100e+01,
         1.100e+01],
        ...,
        [5.760e+03, 7.300e+02, 3.024e+03, ..., 1.000e+01, 1.200e+01,
         1.200e+01],
        [5.760e+03, 9.800e+02, 4.176e+03, ..., 1.200e+01, 1.400e+01,
         1.400e+01],
        [7.000e+00, 7.000e+00, 1.600e+01, ..., 0.000e+00, 0.000e+00,
         0.000e+00]],

       [[4.320e+03, 7.600e+02, 1.872e+03, ..., 7.000e+00, 1.300e+01,
         7.000e+00],
        [4.320e+03, 4.500e+02, 3.744e+03, ..., 8.000e+00, 1.400e+01,
         1.100e+01],
        [4.320e+03, 6.000e+01, 3.168e+03, ..., 7.000e+00, 7.000e+00,
         1.200e+01],
        ...,
        [4.320e+03, 7.500e+02, 3.024e+03, ..., 1.000e+01, 1.300e+01,
         1.300e+01],
        [4.320e+03, 9.800e+02, 4.176e+03, ..., 1.200e+01, 1.400e+01,
   

In [5]:
env_path.shape

(500, 209, 8)

In [14]:
from sklearn.preprocessing import StandardScaler
data = env_path.reshape(env_path.shape[0] * env_path.shape[1],env_path.shape[2])
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized=data_normalized.reshape(env_path.shape[0],1,env_path.shape[1],-1)

In [15]:
data_normalized.shape

(500, 1, 209, 8)

In [8]:
import joblib
joblib.dump(scaler, 'normalized_scaler.pkl')
# scaler = joblib.load('scaler.pkl')

['normalized_scaler.pkl']

## NetWork

In [19]:
import torch.nn as nn
import torch


def conv_block(in_channels, out_channels):
    '''
    returns a block conv-bn-relu-pool
    '''
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels,(30,1), padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

class ProtoNet(nn.Module):

    def __init__(self, x_dim=1, hid_dim=64, z_dim=64):
        super(ProtoNet, self).__init__()
        self.conv1=conv_block(x_dim, hid_dim)
        self.conv2=conv_block(hid_dim, hid_dim)
        self.conv3=conv_block(hid_dim, z_dim)
        self.output = nn.Linear(256,z_dim)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        return self.output(x)
model = ProtoNet() # 40 1 28 28 -> 40 64
print(model(torch.Tensor(1,1,209,8)).shape)

torch.Size([10, 64])


In [24]:
model(torch.Tensor(data_normalized[:10])).shape

torch.Size([10, 64])

In [27]:
from sklearn.preprocessing import LabelEncoder

colors = [8,5,6,6,7,8]
# 创建一个LabelEncoder对象
le = LabelEncoder()

# 对分类变量进行顺序编码
le.fit(colors)
encoded_colors = le.transform(colors)
encoded_colors

array([3, 0, 1, 1, 2, 3])